In [70]:
import pandas as pd
import ast
import numpy as np

In [71]:
movies = pd.read_csv("./Dataset/movies_dataset.csv")
credits = pd.read_csv("./Dataset/credits.csv")

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_24724\954623710.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("./Dataset/movies_dataset.csv")


In [72]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [73]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

### dataset credits

In [74]:
credits["cast"] = credits["cast"].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
credits["crew"] = credits["crew"].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)

In [75]:
credits["cast"] = credits["cast"].apply(lambda x: ', '.join(genre['name'] for genre in x) if isinstance(x, list) else np.nan)
credits["cast"][0]

'Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn, John Ratzenberger, Annie Potts, John Morris, Erik von Detten, Laurie Metcalf, R. Lee Ermey, Sarah Freeman, Penn Jillette'

In [76]:
credits["crew"] = credits["crew"].apply(lambda x: ', '.join(genre['name'] for genre in x if genre["job"]=="Director") if isinstance(x, list)  else np.nan)
credits["crew"][0]

'John Lasseter'

In [77]:
credits["crew"].replace("",np.nan,inplace=True)

### dataset movies

#### Eliminar las columnas que no serán utilizadas: `video, imdb_id, adult, original_title, poster_path y homepage`.


In [78]:
print(movies.shape)
movies.drop(columns=["video","imdb_id","adult","original_title","poster_path" ,"homepage"],inplace=True)
print(movies.shape)

(45466, 24)
(45466, 18)


#### Algunos campos, como `belongs_to_collection`, `production_companies` y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos

In [79]:
#Veo como son los tipos de cada columna anidada
columnas_anidadas = ["belongs_to_collection","genres","production_companies","production_countries","spoken_languages"]
for columna in columnas_anidadas:
    print(columna,": \n", movies[f"{columna}"][0])

belongs_to_collection : 
 {'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}
genres : 
 [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
production_companies : 
 [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries : 
 [{'iso_3166_1': 'US', 'name': 'United States of America'}]
spoken_languages : 
 [{'iso_639_1': 'en', 'name': 'English'}]


#### Notemos que en todos siempre aparece la key: "name" por lo que, solo me voy a quedar con esos datos

In [80]:
for columna in columnas_anidadas:
    movies[f"{columna}"] = movies[f"{columna}"].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
    if columna == "belongs_to_collection":
        belongs_to_collection = pd.json_normalize(movies["belongs_to_collection"])
        movies["belongs_to_collection"] = belongs_to_collection["name"]
    else:
        df = pd.DataFrame()
        df["name"] = movies[f"{columna}"].apply(lambda x: ', '.join(genre['name'] for genre in x) if isinstance(x, list) else np.nan)
        movies[f"{columna}"] = df["name"]

#### Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [81]:
print(movies["revenue"].isna().sum(), movies["budget"].isna().sum())
movies["revenue"].fillna(0,inplace=True)
movies["budget"].fillna(0,inplace=True)
print(movies["revenue"].isna().sum(),movies["budget"].isna().sum())

6 0
0 0


#### Los valores nulos del campo release date deben eliminarse.

In [82]:
print(movies["release_date"].isnull().sum())
movies.dropna(subset=["release_date"], inplace=True)
print(movies["release_date"].isnull().sum())
movies.shape

87
0


(45379, 18)

#### De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [83]:
movies["release_date"] = pd.to_datetime(movies["release_date"],errors='coerce',format="%Y-%m-%d")

In [84]:
movies["release_year"] = movies["release_date"].dt.year

In [85]:
movies["release_year"].isna().sum()

3

#### Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [86]:
movies["budget"] = pd.to_numeric(movies["budget"], errors='coerce')

movies["return"] = movies.apply(lambda row: row["revenue"] / row["budget"] if row["budget"] != 0 else 0, axis=1)

In [87]:
movies["return"].isna().sum()

3

In [88]:
movies["return"].fillna(0,inplace=True)

### Ahora que ya esta limpio

In [89]:
movies["director"] = credits["crew"]

In [90]:
# movies.to_csv("./Dataset/movies_clean.csv")